In [ ]:
"""
<포항시 교통카드 데이터 현황 시각화 분석>
 - 데이터 수집처 : 국가 교통데이터 오픈마켓
 - 수집 URL : bigdata-transportation.kr
 - 검색어 : 포항시 BIS 교통카드 사용내역
 - 회원 가입 필요 합니다.
"""

In [1]:
import pandas as pd
import os

## 데이터 통합 전 -> 한개 파일로 샘플링하기

In [2]:
### 한개 파일 읽어 들이기 : trfcard (0) 폴더의 csv 파일 읽어 들이기
# 변수명 : df_bus_card_org

file_path = "./01_data_org/trfcard (0)/trfcard.csv"
### sep="," : 데이터 구분자 정의 (기본값은 콤마, 생략가능)
df_bus_card_org = pd.read_csv(file_path, sep=",")
df_bus_card_org.head(1)

,on_date,off_date,route_name,descr,age_type,trans_yn,addfee_yn,start_bstop,start_gps_x,start_gps_y,end_bstop,end_gps_x,end_gps_y
0,20200102051049,20200102051844,131,양덕-송도-시청-양학-양덕,일반,N,N,양덕차고지,129.401693,36.077258,동부초등학교,129.380866,36.062209


In [3]:
### 데이터 정보 확인
# - 결측데이터 있는지 확인
df_bus_card_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   on_date      16185 non-null  int64  
 1   off_date     16185 non-null  int64  
 2   route_name   16185 non-null  object 
 3   descr        16185 non-null  object 
 4   age_type     16164 non-null  object 
 5   trans_yn     16185 non-null  object 
 6   addfee_yn    16185 non-null  object 
 7   start_bstop  16185 non-null  object 
 8   start_gps_x  16185 non-null  float64
 9   start_gps_y  16185 non-null  float64
 10  end_bstop    16185 non-null  object 
 11  end_gps_x    16185 non-null  float64
 12  end_gps_y    16185 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 1.6+ MB


In [4]:
df_bus_card_org.describe()

,on_date,off_date,start_gps_x,start_gps_y,end_gps_x,end_gps_y
count,1.618500e+04,1.618500e+04,16185.000000,16185.000000,16185.000000,16185.000000
mean,2.020010e+13,2.020010e+13,129.369118,36.034985,129.368939,36.031927
std,4.127644e+04,4.175148e+04,0.048054,0.043280,0.040777,0.033987
min,2.020010e+13,2.020010e+13,129.052958,35.804119,129.052958,35.804119
25%,2.020010e+13,2.020010e+13,129.350290,36.012537,129.353856,36.013250
50%,2.020010e+13,2.020010e+13,129.365461,36.031720,129.365281,36.034110
75%,2.020010e+13,2.020010e+13,129.383462,36.064088,129.370823,36.046050
max,2.020010e+13,2.020010e+13,129.578137,36.254158,129.578137,36.244762


In [ ]:
"""
<데이터 처리 순서>
 1. 데이터 수집
 2. 샘플 데이터 읽어들여서 정보확인(전처리, 결측, 중복, 이상치 데이터 처리)
 3. 컬럼명 확인 후 변경할지 말지 결정(분석가 마음)
 4. 컬럼들의 조합을 통해 확인 가능한 소주제들 추출(분석가 마음)
 5. 소주제 분석에 필요한 컬럼 생성(가공하기)
 6. 모든 데이터 통합하기(2~5번 반복 수행)
 7. 통합 데이터 파일로 저장하기(원본 폴더와 통합 폴더를 보통 분리해서 저장시킴
 8. 통합 데이터를 이용해서 탐색적 데이터 분석(EDA) -> 보통 시각화와 함께 분석
 ** ETL
"""

## 영문 컬럼 이름을 한글명으로 변경하기

In [ ]:
"""
<컬럼명 변경 순서>
 1. 메타정의서에서 영문 및 한글 데이터가 있는 파일 읽어들이기
 2. 읽어들인 데이터를 "딕셔너리" 형태로 변환
 3. 원본 DataFrame의 컬럼명 변경하기 -> rename(딕셔너리 변수) 함수 사용
"""

In [5]:
# 원본 DataFrame
df_bus_card_org.head(1)

,on_date,off_date,route_name,descr,age_type,trans_yn,addfee_yn,start_bstop,start_gps_x,start_gps_y,end_bstop,end_gps_x,end_gps_y
0,20200102051049,20200102051844,131,양덕-송도-시청-양학-양덕,일반,N,N,양덕차고지,129.401693,36.077258,동부초등학교,129.380866,36.062209


In [6]:
### 메타정의서에서 영문 및 한글 데이터가 있는 파일 읽어들이기
# - 읽어들일 컬럼명 ; 컬럼명(영문), 컬럼명(한글) 2개만 읽어들이기
# - 변수명 : df_bus_card_col_org

file_path = "./01_data_org/trfcard (0)/trfcard_columns.xlsx"
df_bus_card_col_org = pd.read_excel(file_path, header=2, usecols="B:C")
df_bus_card_col_org

df_bus_card_col_org.info()
df_bus_card_col_org

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   컬럼명 (영문)  13 non-null     object
 1   컬럼명 (한글)  13 non-null     object
dtypes: object(2)
memory usage: 336.0+ bytes


,컬럼명 (영문),컬럼명 (한글)
0,on_date,승차시각
1,off_date,하차시각
2,route_name,노선명
3,descr,노선설명
4,age_type,승객연령
5,trans_yn,환승여부
6,addfee_yn,추가운임여부
7,start_bstop,승차정류장
8,start_gps_x,승차정류장 GPS X
9,start_gps_y,승차정류장 GPS Y


In [7]:
### 2. 읽어들인 데이터를 "딕셔너리" 형태로 변환
# - key는 컬럼명(영문)
# - values는 컬럼명(한글)
# - 딕셔너리 변수명 : df_bus_card_col_dict
# - 결과 형태 : {'on_date' : "승차시각", "off_date" : "하차시각", ...}
df_bus_card_col_dict = dict(zip(df_bus_card_col_org["컬럼명 (영문)"], df_bus_card_col_org["컬럼명 (한글)"]))
df_bus_card_col_dict

{'on_date': '승차시각',
 'off_date': '하차시각',
 'route_name': '노선명',
 'descr': '노선설명',
 'age_type': '승객연령',
 'trans_yn': '환승여부',
 'addfee_yn': '추가운임여부',
 'start_bstop': '승차정류장',
 'start_gps_x': '승차정류장 GPS X',
 'start_gps_y': '승차정류장 GPS Y',
 'end_bstop': '하차정류장',
 'end_gps_x': '하차정류장 GPS X',
 'end_gps_y': '하차정류장 GPS Y'}

In [8]:
df_bus_card_col_org["컬럼명 (영문)"][0], df_bus_card_col_org["컬럼명 (한글)"][0]

df_bus_card_col_dict = {}

for idx in range(0, len(df_bus_card_col_org), 1):
    # print(df_bus_card_col_org["컬럼명 (영문)"][idx], df_bus_card_col_org["컬럼명 (한글)"][idx])
    df_bus_card_col_dict[df_bus_card_col_org["컬럼명 (영문)"][idx]] = df_bus_card_col_org["컬럼명 (한글)"][idx]

df_bus_card_col_dict

{'on_date': '승차시각',
 'off_date': '하차시각',
 'route_name': '노선명',
 'descr': '노선설명',
 'age_type': '승객연령',
 'trans_yn': '환승여부',
 'addfee_yn': '추가운임여부',
 'start_bstop': '승차정류장',
 'start_gps_x': '승차정류장 GPS X',
 'start_gps_y': '승차정류장 GPS Y',
 'end_bstop': '하차정류장',
 'end_gps_x': '하차정류장 GPS X',
 'end_gps_y': '하차정류장 GPS Y'}

In [9]:
df_bus_card_col_org["컬럼명 (영문)"][0], df_bus_card_col_org["컬럼명 (한글)"][0]

df_bus_card_col_dict = {}

### zip(리스트1, 리스트2, ... 리스트n) : 모든 리스트의 갯수는 같아야 합니다.
#   - 각 리스트의 인덱스 끼리 하나의 튜플로 묶는 작업을 합니다.
#   - zip(["a", "1"], ["b", "2"], ["c", 3])
#   - 예시(["a", "b", "c"], ["1", "2", "3"]...)
for k, v in zip(df_bus_card_col_org["컬럼명 (영문)"], df_bus_card_col_org["컬럼명 (한글)"]):
    df_bus_card_col_dict[k] = v

df_bus_card_col_dict

{'on_date': '승차시각',
 'off_date': '하차시각',
 'route_name': '노선명',
 'descr': '노선설명',
 'age_type': '승객연령',
 'trans_yn': '환승여부',
 'addfee_yn': '추가운임여부',
 'start_bstop': '승차정류장',
 'start_gps_x': '승차정류장 GPS X',
 'start_gps_y': '승차정류장 GPS Y',
 'end_bstop': '하차정류장',
 'end_gps_x': '하차정류장 GPS X',
 'end_gps_y': '하차정류장 GPS Y'}

In [10]:
df_bus_card_col_dict = {}

### zip(리스트1, 리스트2, ... 리스트n) : 모든 리스트의 갯수는 같아야 합니다.
#   - 각 리스트의 인덱스 끼리 하나의 튜플로 묶는 작업을 합니다.
#   - zip(["a", "1"], ["b", "2"], ["c", 3])
#   - 예시(["a", "b", "c"], ["1", "2", "3"]...)
df_bus_card_col_dict = {k : v for k, v in zip(df_bus_card_col_org["컬럼명 (영문)"], 
                                              df_bus_card_col_org["컬럼명 (한글)"])}

df_bus_card_col_dict

{'on_date': '승차시각',
 'off_date': '하차시각',
 'route_name': '노선명',
 'descr': '노선설명',
 'age_type': '승객연령',
 'trans_yn': '환승여부',
 'addfee_yn': '추가운임여부',
 'start_bstop': '승차정류장',
 'start_gps_x': '승차정류장 GPS X',
 'start_gps_y': '승차정류장 GPS Y',
 'end_bstop': '하차정류장',
 'end_gps_x': '하차정류장 GPS X',
 'end_gps_y': '하차정류장 GPS Y'}

In [11]:
### to_numpy() : Dataframe을 행렬의 리스트 형태로 변환하는 함수
#              : zip()의 구조와 유사
#              : 왼쪽값을 key로, 오른쪽값을 value로 사용함
dict(df_bus_card_col_org.to_numpy())
# df_bus_card_col_dict

{'on_date': '승차시각',
 'off_date': '하차시각',
 'route_name': '노선명',
 'descr': '노선설명',
 'age_type': '승객연령',
 'trans_yn': '환승여부',
 'addfee_yn': '추가운임여부',
 'start_bstop': '승차정류장',
 'start_gps_x': '승차정류장 GPS X',
 'start_gps_y': '승차정류장 GPS Y',
 'end_bstop': '하차정류장',
 'end_gps_x': '하차정류장 GPS X',
 'end_gps_y': '하차정류장 GPS Y'}

In [12]:
### 3. 원본 DataFrame의 컬럼명 변경하기 -> rename(columns=딕셔너리 변수) 함수 사용
# - 현재까지 원본 프레임 변수명 : df_bus_card_org
# - 결과 데이터프레임 변수명 : df_bus_card_kor
df_bus_card_kor = df_bus_card_org.rename(columns=df_bus_card_col_dict)
df_bus_card_kor.head(1)

,승차시각,하차시각,노선명,노선설명,승객연령,환승여부,추가운임여부,승차정류장,승차정류장 GPS X,승차정류장 GPS Y,하차정류장,하차정류장 GPS X,하차정류장 GPS Y
0,20200102051049,20200102051844,131,양덕-송도-시청-양학-양덕,일반,N,N,양덕차고지,129.401693,36.077258,동부초등학교,129.380866,36.062209


## 4. 컬럼들의 조합을 통해 확인 가능한 소주제들 추출(분석가 마음)

In [13]:
### 데이터 탐색
# - unique() : 고유한 값만 추출
df_bus_card_kor["승객연령"].unique()

array(['일반', '청소년', nan, '어린이'], dtype=object)

In [ ]:
"""
<주제 도출>
1. 버스 이용량 분석(필요한 컬럼 : "연도, 월, 일, 시간, 분", 승객연령, count 집계)
 - 연간 버스 이용량 분석
 - 월별 vs 일자별 버스 이용량 현황 분석
 - 일자별 vs 시간별 버스 이용량 현황 분석
 - 시간별 vs 분별 버스 이용량 현황 분석

2. 버스 내 체류시간 분석 (필요한 컬럼 : "체류 시간")
 - 체류시간 = 하차시각 - 승차시각

3. 버스 승하차 구간별 버스내 체류시간 분석(필요한 컬럼 : "승하차구간")
 - 체류시간이 높은 구간을 기준으로 내림차수하여 급행 도입이 가능한 구간 확인

## 위 소주제별로 필요한 컬럼들을 Dataframe에 컬럼을 추가하여 사용(통합을 위해서)
"""

In [14]:
df_bus_card_kor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   승차시각         16185 non-null  int64  
 1   하차시각         16185 non-null  int64  
 2   노선명          16185 non-null  object 
 3   노선설명         16185 non-null  object 
 4   승객연령         16164 non-null  object 
 5   환승여부         16185 non-null  object 
 6   추가운임여부       16185 non-null  object 
 7   승차정류장        16185 non-null  object 
 8   승차정류장 GPS X  16185 non-null  float64
 9   승차정류장 GPS Y  16185 non-null  float64
 10  하차정류장        16185 non-null  object 
 11  하차정류장 GPS X  16185 non-null  float64
 12  하차정류장 GPS Y  16185 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 1.6+ MB


## 소수제들에 필요한 컬럼 추가하기

In [ ]:
### 추가 컬럼 : 년, 월, 일, 시간, 분, 승객연령, 체류시간, 승하차군가(승차+하차정류장)

In [ ]:
### 년, 월, 일, 시간, 분을 -> 승차시간 데이터에서 추출하기
# - 승차시각 데이터 타입이 int이므로 -> 추출을 위해 str타입으로 변환하기
#                                    -> str 타입을 다시 날짜타입으로 변환하여 필요한 값 추출
# - 변수 타입 변경(형변환) 함수 : int(), float(), str(), list(), dict(), tuple()
# - 데이터프레임의 데이터 타입 변경 함수 : astype()

In [15]:
### 승차시각 데이터를 문자 타입(object)으로 변환하기
df_bus_card_kor = df_bus_card_kor.astype({"승차시각" : str, "하차시각" : str})
df_bus_card_kor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17247 entries, 0 to 17246
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   승차시각         17247 non-null  object 
 1   하차시각         17247 non-null  object 
 2   노선명          17247 non-null  object 
 3   노선설명         17247 non-null  object 
 4   승객연령         17229 non-null  object 
 5   환승여부         17247 non-null  object 
 6   추가운임여부       17247 non-null  object 
 7   승차정류장        17247 non-null  object 
 8   승차정류장 GPS X  17247 non-null  float64
 9   승차정류장 GPS Y  17247 non-null  float64
 10  하차정류장        17247 non-null  object 
 11  하차정류장 GPS X  17247 non-null  float64
 12  하차정류장 GPS Y  17247 non-null  float64
dtypes: float64(4), object(9)
memory usage: 1.7+ MB


In [15]:
### 승하차시각 데이터를 날짜 타입으로 변환하기
# - 날짜 타입으로 변환할 떄는 형변환 형태가 아닌,
#   날짜 라이브러리를 이용해서 값을 변환해서 넣어주어야 합니다.
# - 날짜 라이브러리 : Pandas에서 제공하는 to_datetime() 함수 사용
# - 날짜 포멧 형태 : 전제 조건은 문자열인 경우에...
#                  : yyyymmddhhmss, yyyy-mm-dd hh:mm:ss, yyyy.mm.dd hh:mm:dd, 
#                  : yymmddhhmmss, yyyy/mm/dd hh:mm:ss
df_bus_card_kor["승차시각"] = pd.to_datetime(df_bus_card_kor["승차시각"])
df_bus_card_kor["하차시각"] = pd.to_datetime(df_bus_card_kor["하차시각"])
df_bus_card_kor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   승차시각         16185 non-null  datetime64[ns]
 1   하차시각         16185 non-null  datetime64[ns]
 2   노선명          16185 non-null  object        
 3   노선설명         16185 non-null  object        
 4   승객연령         16164 non-null  object        
 5   환승여부         16185 non-null  object        
 6   추가운임여부       16185 non-null  object        
 7   승차정류장        16185 non-null  object        
 8   승차정류장 GPS X  16185 non-null  float64       
 9   승차정류장 GPS Y  16185 non-null  float64       
 10  하차정류장        16185 non-null  object        
 11  하차정류장 GPS X  16185 non-null  float64       
 12  하차정류장 GPS Y  16185 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 1.6+ MB


In [16]:
### 원본 사용하지 않고, 새로운 데이터프레임에 필요한 컬럼들만 별도 가공
# - 변수명 : df_bus_card
# - 별도 관리 컬럼 : 승차시간, 하차시각, 승객연령, 승차정류장, 하차정류장
# - copy() : 원본 데이터에서 추출해서 정보를 가져 올 때 사용하자
df_bus_card = df_bus_card_kor[["승차시각", "하차시각", "승객연령", "승차정류장", "하차정류장"]].copy()
df_bus_card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   승차시각    16185 non-null  datetime64[ns]
 1   하차시각    16185 non-null  datetime64[ns]
 2   승객연령    16164 non-null  object        
 3   승차정류장   16185 non-null  object        
 4   하차정류장   16185 non-null  object        
dtypes: datetime64[ns](2), object(3)
memory usage: 632.4+ KB


In [17]:
### 년, 월, 일, 시, 분 컬럼 추가하기
df_bus_card["기준년도"] = df_bus_card["승차시각"].dt.year
df_bus_card["기준월"] = df_bus_card["승차시각"].dt.month
df_bus_card["기준일"] = df_bus_card["승차시각"].dt.day
df_bus_card["기준시간"] = df_bus_card["승차시각"].dt.hour
df_bus_card["기준분"] = df_bus_card["승차시각"].dt.minute

df_bus_card.head(5)
df_bus_card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   승차시각    16185 non-null  datetime64[ns]
 1   하차시각    16185 non-null  datetime64[ns]
 2   승객연령    16164 non-null  object        
 3   승차정류장   16185 non-null  object        
 4   하차정류장   16185 non-null  object        
 5   기준년도    16185 non-null  int32         
 6   기준월     16185 non-null  int32         
 7   기준일     16185 non-null  int32         
 8   기준시간    16185 non-null  int32         
 9   기준분     16185 non-null  int32         
dtypes: datetime64[ns](2), int32(5), object(3)
memory usage: 948.5+ KB


In [48]:
### "버스내체류시간(분)" 컬럼 추가하기
# - (하차시각 - 승차시각)을 분단위로 계산해서 처리
#    -> 뺀값을 초단위로 환산 -> 60으로 나눠주기
df_bus_card["버스내체류시간(분)"] = round((df_bus_card["하차시각"] - df_bus_card["승차시각"]).dt.total_seconds() / 60, 2)
df_bus_card.head(5)


,승차시각,하차시각,승객연령,승차정류장,하차정류장,기준년도,기준월,기준일,기준시간,기준분,버스내체류시간(분),승하차구간
0,2020-03-31 05:24:33,2020-03-31 06:03:03,일반,문덕사거리,오거리,2020,3,31,5,24,38.50,문덕사거리 → 오거리
1,2020-03-31 05:30:58,2020-03-31 05:35:55,일반,북구보건소,포항세관,2020,3,31,5,30,4.95,북구보건소 → 포항세관
2,2020-03-31 05:38:13,2020-03-31 05:46:16,일반,이동이마트,시외버스터미널,2020,3,31,5,38,8.05,이동이마트 → 시외버스터미널
3,2020-03-31 05:40:32,2020-03-31 06:03:31,일반,연일읍사무소,죽도시장,2020,3,31,5,40,22.98,연일읍사무소 → 죽도시장
4,2020-03-31 05:41:53,2020-03-31 05:52:36,일반,두호초등학교,중앙상가,2020,3,31,5,41,10.72,두호초등학교 → 중앙상가


In [19]:
### "승하차구간" 컬럼 추가하기
# - 승하차구간 = 승차정류자 + "->" + 하차정류장
df_bus_card["승하차구간"] = df_bus_card_kor["승차정류장"] + " → " + df_bus_card_kor["하차정류장"]
df_bus_card.head(5)

,승차시각,하차시각,승객연령,승차정류장,하차정류장,기준년도,기준월,기준일,기준시간,기준분,버스내체류시간(분),승하차구간
0,1970-01-01 05:36:40.102051049,1970-01-01 05:36:40.102051844,일반,양덕차고지,동부초등학교,1970,1,1,5,36,0.0,양덕차고지 → 동부초등학교
1,1970-01-01 05:36:40.102051338,1970-01-01 05:36:40.102054549,일반,문덕사거리,새마을금고해도지점,1970,1,1,5,36,0.0,문덕사거리 → 새마을금고해도지점
2,1970-01-01 05:36:40.102051549,1970-01-01 05:36:40.102051930,일반,문덕사거리,성우오토모티브,1970,1,1,5,36,0.0,문덕사거리 → 성우오토모티브
3,1970-01-01 05:36:40.102051617,1970-01-01 05:36:40.102055046,일반,부영사랑3차,오거리,1970,1,1,5,36,0.0,부영사랑3차 → 오거리
4,1970-01-01 05:36:40.102051841,1970-01-01 05:36:40.102052310,일반,문덕온천,용덕사거리,1970,1,1,5,36,0.0,문덕온천 → 용덕사거리


In [20]:
### 기준년도가 몇년치 데이터가 있는지 확인하기(고유한값 확인)
# - 파일 모두 통합 후에 -> 다시한번 연도는 확인해 봐야 합니다.
# - 연도가 1개년도만 있는 경우에는 연도별 비교 대상이 없으므로, 분석에서 제외
df_bus_card["기준년도"].unique()

array([1970], dtype=int32)

In [21]:
df_bus_card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16185 entries, 0 to 16184
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   승차시각        16185 non-null  datetime64[ns]
 1   하차시각        16185 non-null  datetime64[ns]
 2   승객연령        16164 non-null  object        
 3   승차정류장       16185 non-null  object        
 4   하차정류장       16185 non-null  object        
 5   기준년도        16185 non-null  int32         
 6   기준월         16185 non-null  int32         
 7   기준일         16185 non-null  int32         
 8   기준시간        16185 non-null  int32         
 9   기준분         16185 non-null  int32         
 10  버스내체류시간(분)  16185 non-null  float64       
 11  승하차구간       16185 non-null  object        
dtypes: datetime64[ns](2), float64(1), int32(5), object(4)
memory usage: 1.2+ MB


## 모든 파일 하나의 Dataframe으로 통합하기

In [ ]:
### 통합 후 총 데이터 건수 : 842,608건
# - 통합에 사용 할 DataFrame 변수명 : df_bus_card_tot
# - 꺠끗한 데이터프레임 생성 : pd.DataFrame()


In [30]:
# 통합할 DataFrame 정의
df_bus_card_tot = pd.DataFrame()

for i in range (0, 81, 1) :
    file_path = f"./01_data_org/trfcard ({i})/trfcard.csv"

    try :
        df_bus_card_org = pd.read_csv(file_path, sep=",")

        # 컬럼명 영문에서 한글로 변경하기
        file_path = f"./01_data_org/trfcard ({i})/trfcard_columns.xlsx"
        df_bus_card_col_org = pd.read_excel(file_path, header=2, usecols="B:C")

        dict(df_bus_card_col_org.to_numpy())

        df_bus_card_kor = df_bus_card_org.rename(columns=df_bus_card_col_dict)

        # 소수제 컬럼 추가하기
        df_bus_card_kor = df_bus_card_kor.astype({"승차시각" : str, "하차시각" : str})

        df_bus_card_kor["승차시각"] = pd.to_datetime(df_bus_card_kor["승차시각"])
        df_bus_card_kor["하차시각"] = pd.to_datetime(df_bus_card_kor["하차시각"])
        
        df_bus_card = df_bus_card_kor[["승차시각", "하차시각", "승객연령", "승차정류장", "하차정류장"]].copy()

        # 년, 월, 일, 시, 분 추가
        df_bus_card["기준년도"] = df_bus_card["승차시각"].dt.year
        df_bus_card["기준월"] = df_bus_card["승차시각"].dt.month
        df_bus_card["기준일"] = df_bus_card["승차시각"].dt.day
        df_bus_card["기준시간"] = df_bus_card["승차시각"].dt.hour
        df_bus_card["기준분"] = df_bus_card["승차시각"].dt.minute

        # 버스 내 체류시간 추가
        df_bus_card["버스내체류시간(분)"] = round((df_bus_card["하차시각"] - df_bus_card["승차시각"]).dt.total_seconds() / 60, 2)

        # 승하차구간 추가
        df_bus_card["승하차구간"] = df_bus_card_kor["승차정류장"] + " → " + df_bus_card_kor["하차정류장"]

        df_bus_card_tot = pd.concat([df_bus_card_tot, df_bus_card], ignore_index=True)

    except :
        break

In [40]:
root = os.getcwd()
dir_nm = os.listdir(root)[1]
file_list = os.listdir(dir_nm)
df_bus_card_tot = pd.DataFrame()
for file_nm in file_list :
    file_path = f"./{dir_nm}/{file_nm}/trfcard.csv"
    
    try :
        df_bus_card_org = pd.read_csv(file_path, sep=",")

        # 컬럼명 영문에서 한글로 변경하기
        file_path = f"./01_data_org/trfcard (0)/trfcard_columns.xlsx"
        df_bus_card_col_org = pd.read_excel(file_path, header=2, usecols="B:C")

        dict(df_bus_card_col_org.to_numpy())

        df_bus_card_kor = df_bus_card_org.rename(columns=df_bus_card_col_dict)

        # 소수제 컬럼 추가하기
        df_bus_card_kor = df_bus_card_kor.astype({"승차시각" : str, "하차시각" : str})

        df_bus_card_kor["승차시각"] = pd.to_datetime(df_bus_card_kor["승차시각"])
        df_bus_card_kor["하차시각"] = pd.to_datetime(df_bus_card_kor["하차시각"])
        
        df_bus_card = df_bus_card_kor[["승차시각", "하차시각", "승객연령", "승차정류장", "하차정류장"]].copy()

        # 년, 월, 일, 시, 분 추가
        df_bus_card["기준년도"] = df_bus_card["승차시각"].dt.year
        df_bus_card["기준월"] = df_bus_card["승차시각"].dt.month
        df_bus_card["기준일"] = df_bus_card["승차시각"].dt.day
        df_bus_card["기준시간"] = df_bus_card["승차시각"].dt.hour
        df_bus_card["기준분"] = df_bus_card["승차시각"].dt.minute

        # 버스 내 체류시간 추가
        df_bus_card["버스내체류시간(분)"] = round((df_bus_card["하차시각"] - df_bus_card["승차시각"]).dt.total_seconds() / 60, 2)

        # 승하차구간 추가
        df_bus_card["승하차구간"] = df_bus_card_kor["승차정류장"] + " → " + df_bus_card_kor["하차정류장"]

        df_bus_card_tot = pd.concat([df_bus_card_tot, df_bus_card], ignore_index=True)

    except :
        break

In [43]:
### 통합할 데이터프레임 정의
df_bus_card_tot = pd.DataFrame()

### 메타정의서 파일 불러들이기 (모두 동일하기에 최초 한번만 수행)
file_path = f"./01_data_org/trfcard (0)/trfcard_columns.xlsx"
df_bus_card_col_org = pd.read_excel(file_path, header=2, usecols="B:C")

### 모든 폴더의 파일 불러들이면서 통합 처리하기(반복 80번)
for i in range (0, 80, 1) :
    print(f"[{i}]번째 처리 중.....")
    
    # 컬럼명 영문에서 한글로 변경하기
    file_path = f"./01_data_org/trfcard ({i})/trfcard.csv"
    df_bus_card_org = pd.read_csv(file_path, sep=",")
  

    dict(df_bus_card_col_org.to_numpy())

    df_bus_card_kor = df_bus_card_org.rename(columns=df_bus_card_col_dict)
    # -
    df_bus_card_kor = df_bus_card_kor.astype({"승차시각" : str, "하차시각" : str})
    # - 날짜 타입으로 변환
    df_bus_card_kor["승차시각"] = pd.to_datetime(df_bus_card_kor["승차시각"])
    df_bus_card_kor["하차시각"] = pd.to_datetime(df_bus_card_kor["하차시각"])

    ### 분석에 사용 할 DataFrame 새로 구성
    df_bus_card = df_bus_card_kor[["승차시각", "하차시각", "승객연령", "승차정류장", "하차정류장"]].copy()

    # 년, 월, 일, 시, 분 컬럼 추가
    df_bus_card["기준년도"] = df_bus_card["승차시각"].dt.year
    df_bus_card["기준월"] = df_bus_card["승차시각"].dt.month
    df_bus_card["기준일"] = df_bus_card["승차시각"].dt.day
    df_bus_card["기준시간"] = df_bus_card["승차시각"].dt.hour
    df_bus_card["기준분"] = df_bus_card["승차시각"].dt.minute

    # 버스 내 체류시간 컬럼 추가
    df_bus_card["버스내체류시간(분)"] = round((df_bus_card["하차시각"] - df_bus_card["승차시각"]).dt.total_seconds() / 60, 2)

    # 승하차구간 컬럼 추가
    df_bus_card["승하차구간"] = df_bus_card_kor["승차정류장"] + " → " + df_bus_card_kor["하차정류장"]

    df_bus_card_tot = pd.concat([df_bus_card_tot, df_bus_card], ignore_index=True)

print("통합이 완료 되었습니다.")

[0]번째 처리 중.....
[1]번째 처리 중.....
[2]번째 처리 중.....
[3]번째 처리 중.....
[4]번째 처리 중.....
[5]번째 처리 중.....
[6]번째 처리 중.....
[7]번째 처리 중.....
[8]번째 처리 중.....
[9]번째 처리 중.....
[10]번째 처리 중.....
[11]번째 처리 중.....
[12]번째 처리 중.....
[13]번째 처리 중.....
[14]번째 처리 중.....
[15]번째 처리 중.....
[16]번째 처리 중.....
[17]번째 처리 중.....
[18]번째 처리 중.....
[19]번째 처리 중.....
[20]번째 처리 중.....
[21]번째 처리 중.....
[22]번째 처리 중.....
[23]번째 처리 중.....
[24]번째 처리 중.....
[25]번째 처리 중.....
[26]번째 처리 중.....
[27]번째 처리 중.....
[28]번째 처리 중.....
[29]번째 처리 중.....
[30]번째 처리 중.....
[31]번째 처리 중.....
[32]번째 처리 중.....
[33]번째 처리 중.....
[34]번째 처리 중.....
[35]번째 처리 중.....
[36]번째 처리 중.....
[37]번째 처리 중.....
[38]번째 처리 중.....
[39]번째 처리 중.....
[40]번째 처리 중.....
[41]번째 처리 중.....
[42]번째 처리 중.....
[43]번째 처리 중.....
[44]번째 처리 중.....
[45]번째 처리 중.....
[46]번째 처리 중.....
[47]번째 처리 중.....
[48]번째 처리 중.....
[49]번째 처리 중.....
[50]번째 처리 중.....
[51]번째 처리 중.....
[52]번째 처리 중.....
[53]번째 처리 중.....
[54]번째 처리 중.....
[55]번째 처리 중.....
[56]번째 처리 중.....
[57]번째 처리 중.....
[58]번째 처리 중.....
[59]번째 

## 통합한 Dataframe 파일로 저장

In [ ]:
### 저장 폴더명 : 02_data_all
#   저장 파일명 : df_bus_card_tot.csv

In [44]:
len(df_bus_card_tot)

842608

In [45]:
df_bus_card_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842608 entries, 0 to 842607
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   승차시각        842608 non-null  datetime64[ns]
 1   하차시각        842608 non-null  datetime64[ns]
 2   승객연령        841683 non-null  object        
 3   승차정류장       842608 non-null  object        
 4   하차정류장       842608 non-null  object        
 5   기준년도        842608 non-null  int32         
 6   기준월         842608 non-null  int32         
 7   기준일         842608 non-null  int32         
 8   기준시간        842608 non-null  int32         
 9   기준분         842608 non-null  int32         
 10  버스내체류시간(분)  842608 non-null  float64       
 11  승하차구간       842608 non-null  object        
dtypes: datetime64[ns](2), float64(1), int32(5), object(4)
memory usage: 61.1+ MB


In [28]:
df_bus_card_tot.head(1)

,승차시각,하차시각,승객연령,승차정류장,하차정류장,기준년도,기준월,기준일,기준시간,기준분,버스내체류시간(분),승하차구간
0,2020-01-02 05:10:49,2020-01-02 05:18:44,일반,양덕차고지,동부초등학교,2020,1,2,5,10,7.92,양덕차고지 → 동부초등학교


In [46]:
save_path = "./02_data_all/df_bus_card_tot.csv"
df_bus_card_tot.to_csv(save_path, index=False)

In [47]:
### 저장된 파일 읽어들여서 ㅎ다시 확인
pd.read_csv(save_path).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842608 entries, 0 to 842607
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   승차시각        842608 non-null  object 
 1   하차시각        842608 non-null  object 
 2   승객연령        841683 non-null  object 
 3   승차정류장       842608 non-null  object 
 4   하차정류장       842608 non-null  object 
 5   기준년도        842608 non-null  int64  
 6   기준월         842608 non-null  int64  
 7   기준일         842608 non-null  int64  
 8   기준시간        842608 non-null  int64  
 9   기준분         842608 non-null  int64  
 10  버스내체류시간(분)  842608 non-null  float64
 11  승하차구간       842608 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 77.1+ MB
